# Goal

to fetch the events from the calendar and edit them as a timebox given prompt and then write them back.

That way we can give this as a tool to the schedular agent as well, so it can change a single event, but also reschedule the day as a whole if needed.

In [1]:
from datetime import datetime, timedelta
work_start_hour = 8 # TODO: move to settings
is_before_work_start = datetime.now().time() < datetime.now().replace(hour=work_start_hour, minute=0, second=0, microsecond=0).time()
# is_before_work_start = True
today = datetime.now().date()
day_to_plan = today if is_before_work_start else today + timedelta(days=1)
day_to_plan

datetime.date(2026, 1, 31)

# now pull events from calendar for that date as a timebox object

```mermaid

flowchart TB
  node_1("CalendarEvent")

```

In [ ]:
from dataclasses import dataclass
from datetime import datetime, date as date_type, time as time_type, timedelta, timezone
from typing import Any, Iterable
from zoneinfo import ZoneInfo

from dateutil import parser as date_parser

from fateforger.agents.schedular.models.calendar import CalendarEvent, EventType
from fateforger.agents.timeboxing.timebox import Timebox


from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams

from fateforger.core.config import settings


# TODO: wrao this in a nice function

def calendar_workbench() -> McpWorkbench:

    return McpWorkbench(
        StreamableHttpServerParams(url=settings.mcp_calendar_server_url, timeout=10.0)
    )
workbench = calendar_workbench()

# query the workbench
result = await workbench.call_tool(
    "list-events", # TODO: what are all the args we can pass here?
    arguments={
        "calendarId": "primary",
        "timeMin": "2026-01-27T00:00:00Z",
        "timeMax": "2026-01-28T00:00:00Z",
        "singleEvents": True,
        "orderBy": "startTime",
    },
)

raw_json_string = result.result[0].content

# def get_timebox_for_day(day: date_type, timezone_str: str) -> Timebox:

In [2]:
result.result[0].content

'{"events":[{"id":"30287b8c333b44c18b3bfbd52f357d4f","summary":"morning review","start":{"dateTime":"2026-01-27T13:00:00+01:00","timeZone":"Europe/Amsterdam"},"end":{"dateTime":"2026-01-27T13:15:00+01:00","timeZone":"Europe/Amsterdam"},"status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt","created":"2026-01-26T21:07:30.000Z","updated":"2026-01-27T10:54:23.636Z","colorId":"10","creator":{"email":"hugo.evers@gmail.com","self":true},"organizer":{"email":"hugo.evers@gmail.com","self":true},"iCalUID":"30287b8c333b44c18b3bfbd52f357d4f@google.com","sequence":1,"reminders":{"useDefault":true},"eventType":"default","guestsCanModify":true,"calendarId":"primary","accountId":"normal"}],"totalCount":1}'

In [3]:
from fateforger.adapters.calendar.models import GCalEventsResponse

gcal_events = GCalEventsResponse.model_validate_json(result.result[0].content)

In [4]:
gcal_events

GCalEventsResponse(events=[GCalEvent(id='30287b8c333b44c18b3bfbd52f357d4f', summary='morning review', start=GCalEventDateTime(date_time='2026-01-27T13:00:00+01:00', date=None, time_zone='Europe/Amsterdam'), end=GCalEventDateTime(date_time='2026-01-27T13:15:00+01:00', date=None, time_zone='Europe/Amsterdam'), status='confirmed', html_link='https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt', created='2026-01-26T21:07:30.000Z', updated='2026-01-27T10:54:23.636Z', creator=GCalPerson(email='hugo.evers@gmail.com', self_=True), organizer=GCalPerson(email='hugo.evers@gmail.com', self_=True), ical_uid='30287b8c333b44c18b3bfbd52f357d4f@google.com', sequence=1, reminders=GCalReminders(use_default=True, overrides=None), event_type='default', guests_can_modify=True, calendar_id='primary', account_id='normal', colorId='10')], total_count=1)

what is the goal here?
we want to give the agent a tool so it can pull the timebox for a given day.

we want to take that json object and turn it into a pydantic object.

# TODO: link to code

now we want to have a model that we use for timeboxing, so we convert it into a timebox object, l

In [ ]:
# import from src/fateforger/adapters/calendar/models.py


NameError: name 'raw_json_string' is not defined

In [6]:
from fateforger.agents.schedular.models.calendar import CalendarEvent



In [7]:
# now let the agent do it
# agent needs to know which day to fetch and the timezone, it can default to users.info for the timezone.
# and assume the user wants to plan for the next work day if before work start hour, unless the user specifies otherwise.


# Inspecting the timebox object

does it have all the properties/aliases to match the calendar event objects so we can mix and match directly?
Does the timebox object have the serialisation methods to inject it into the agents' context?

# now apply json patches

for this part we must determine what to prompt the model, so its going to generate json patches, but in which model? it needs to use the minimized schema, it will need to know about the current timebox object, it needs instructions, the inputs.

initially we dont want it to use issue json patches to create a timebox, instead it should print in markdown a rough draft of the timebox, which we can then convert to json patches to create the timebox object.
the question is if we assume the user agreees with the rought draft whether we can create the first real timebox.

In [ ]:
# the timeboxing agent should have as it memory the timebox object, the inputs, and the last user message

# stage 3: rough draft of timebox

## TODO: 
* make sure the constraints and inputs are passed correctly to the rough draft agent, this means also injecting the current timebox events as fixed events.
* parametrize the planning rules and draft algorithm
* validate the markdown output and the planning trace output
* make sure we can insert this is toon input into the stage 4 timebox json patching agent

```xml
<SystemPrompt>
  <Role>Rough-Draft Day Planner Agent</Role>

  <Objective>
    Generate a bird’s-eye, glanceable rough draft of a day plan from user constraints and inputs.
    The output is meant to be refined in later stages; prioritize clarity, order, and approximate durations.
  </Objective>

  <Inputs>
    <Constraints>
      Fixed events with exact start/end (sleep, meetings, travel).
      Day start time (e.g., work starts 08:00).
      Preferred block sizes (e.g., DW=2h, half-DW=1h, quarter-DW=30m).
      Preferences (e.g., gym early, stimulant window ~4h).
    </Constraints>
    <Tasks>
      DailyOneThing (single most important outcome).
      Supporting tasks (meal prep, chores, fun/creative time).
      Optional constraints about stimulant timing, energy, or sequencing.
    </Tasks>
  </Inputs>

  <PlanningRules>
    <Rule id="R1">Honor all fixed-time constraints exactly; do not overlap events.</Rule>
    <Rule id="R2">Place DailyOneThing into 2–3 Deep Work blocks early-to-midday unless constraints prevent it.</Rule>
    <Rule id="R3">Default DW blocks to the user’s standard duration; allow 1h or 30m “partial DW” if specified.</Rule>
    <Rule id="R4">Insert buffers where risk is high: after DW (10–15m), after gym (30–45m reset), after meals (10–20m).</Rule>
    <Rule id="R5">Keep Shallow Work ≤30% of waking hours; cluster it later in the day when possible.</Rule>
    <Rule id="R6">Protect recovery and fun/creative blocks as first-class events; do not label them “optional” unless user did.</Rule>
    <Rule id="R7">If stimulants are mentioned: align demanding DW within the effective window; avoid back-to-back dosing unless user insists.</Rule>
    <Rule id="R8">Prefer simple sequencing over precision. This is a rough draft for later refinement.</Rule>
  </PlanningRules>

  <DraftAlgorithm>
    <Step>Lock fixed anchors (sleep, immovable meetings, hard start time).</Step>
    <Step>Place Morning Routine immediately after sleep end (15–45m, per user preference).</Step>
    <Step>Schedule DW blocks for DailyOneThing: hardest block first, then execution/polish blocks.</Step>
    <Step>Place gym in the user’s preferred slot (e.g., early) and add a reset buffer after.</Step>
    <Step>Place a fun/unstructured block (1–2h) if requested, ideally between stimulant windows or after a major block.</Step>
    <Step>Place shallow work (chores/admin) later; keep it bounded.</Step>
    <Step>Place meal prep + dinner in the evening; then creative time if requested.</Step>
    <Step>End with shutdown ritual + reading in bed + sleep anchor.</Step>
  </DraftAlgorithm>

  <OutputSpec>
    <PrimaryOutput format="markdown">
      <![CDATA[
      Produce a short, glanceable summary with:
      - A few headings (Night/Morning/Midday/Afternoon/Evening) OR emojis (optional).
      - ONE line per major chunk (not per micro-event).
      - Each line: **Label** — rough duration (e.g., "Deep Work (primary) — 2h").
      - Do not include exact timestamps unless the user explicitly asked for times.
      - Keep total output under ~12–15 lines.
      ]]>
    </PrimaryOutput>

    <PlanningTrace format="xml">
      <TraceRules>
        <Rule>Do NOT include chain-of-thought or detailed internal reasoning.</Rule>
        <Rule>Include only: assumptions, placements, durations, and checks.</Rule>
      </TraceRules>
      <TraceTemplate>
        <![CDATA[
        <PlanTrace>
          <Assumptions>
            <Assumption id="A1">...</Assumption>
          </Assumptions>
          <Placements>
            <Placement type="DW" label="DailyOneThing" duration="PT2H" rationale="hardest first"/>
            <Placement type="H" label="Gym" duration="PT1H30M" rationale="early preference"/>
          </Placements>
          <Checks>
            <Check name="NoOverlaps" status="pass"/>
            <Check name="ShallowWorkRatio" status="pass|warn" value="..."/>
            <Check name="DWCount" status="pass|warn" value="..."/>
          </Checks>
        </PlanTrace>
        ]]>
      </TraceTemplate>
    </PlanningTrace>

    <Tone>
      Crisp, minimal, readable at a glance. No motivational speeches.
    </Tone>
  </OutputSpec>

  <FailureModes>
    <Mode id="F1">Missing fixed anchors: infer a reasonable sleep block only if user provided sleep duration + bedtime; otherwise ask one question.</Mode>
    <Mode id="F2">Too many tasks: prioritize DailyOneThing and cap DW at 3 blocks; push the rest to shallow work or mark as overflow.</Mode>
    <Mode id="F3">Timing conflict: preserve fixed events, then reflow flexible blocks; keep buffers if possible.</Mode>
  </FailureModes>
</SystemPrompt>

```

In [5]:
from llama_index.tools.artifact_editor import (
    ArtifactEditorToolSpec,
    ArtifactMemoryBlock,
)

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

token = os.getenv("OPENROUTER_API_KEY")
open_router_base_url = os.getenv("OPENROUTER_BASE_URL")
client = OpenAI(api_key=token, base_url=open_router_base_url)



In [ ]:
from pydantic import BaseModel
from typing import Literal, Optional

class PizzaOrder(BaseModel):
    size: Literal["small", "medium", "large", "extra-large"]
    toppings: list[str]
    address: str


schema = PizzaOrder.model_json_schema()

prompt = lambda order: f"""
You are an expert pizza ordering agent. You need to order a pizza for the user based on their preferences.
you respond with json and list the size, toppings and address to deliver to.
Here is the order:
{order}
"""

order = "I want a large pizza with pepperoni and mushrooms, deliver to 123 Main St."
model="google/gemini-3-flash-preview"
response = client.responses.create(
  model=model,
  input=[{"role": "user", "content": prompt(order)}],
  text={
    "format": {
      "type": "json_schema",
      "name": "shopping_list",
      "strict": True,
      "schema": schema
    }
  }
)
pizza_order = PizzaOrder.model_validate_json(response.output_text)
pizza_order

PizzaOrder(size='medium', toppings=['cheese'], address='1600 Pennsylvania Avenue NW, Washington, DC 20500')

In [40]:
from typing import Annotated, Literal, Optional, Union
from pydantic import BaseModel, Field

class Pizza(BaseModel):
    name: str = Field(description="The name of the pizza")
    remove: Optional[list[str]] = Field(
        default=None,
        description="Ingredients the customer requests to remove",
    )
    add: Optional[list[str]] = Field(
        default=None,
        description="Ingredients the customer requests to add",
    )

class PizzaOrderDoc(BaseModel):
    pizzas: list[Pizza] = Field(default_factory=list)

class PizzaUpdate(BaseModel):
    name: Optional[str] = None
    remove: Optional[list[str]] = None
    add: Optional[list[str]] = None

class AddPizzaOp(BaseModel):
    op: Literal["add"] = "add"
    index: int | Literal["-"] = "-"
    value: Pizza

class UpdatePizzaOp(BaseModel):
    op: Literal["update"] = "update"
    index: int
    value: PizzaUpdate

class RemovePizzaOp(BaseModel):
    op: Literal["remove"] = "remove"
    index: int

class MovePizzaOp(BaseModel):
    op: Literal["move"] = "move"
    from_index: int
    to_index: int

PizzaEditOp = Annotated[
    Union[AddPizzaOp, UpdatePizzaOp, RemovePizzaOp, MovePizzaOp],
    Field(discriminator="op"),
]

class PizzaEditPlan(BaseModel):
    operations: list[PizzaEditOp] = Field(default_factory=list)


def _validate_index(index: int, size: int, *, allow_end: bool = False) -> None:
    max_index = size if allow_end else size - 1
    if index < 0 or index > max_index:
        raise IndexError(f"Index {index} out of range for list size {size}.")


def apply_pizza_plan(doc: PizzaOrderDoc, plan: PizzaEditPlan) -> PizzaOrderDoc:
    pizzas = list(doc.pizzas)
    for op in plan.operations:
        if isinstance(op, AddPizzaOp):
            if op.index == "-":
                pizzas.append(op.value)
            else:
                _validate_index(op.index, len(pizzas), allow_end=True)
                pizzas.insert(op.index, op.value)
        elif isinstance(op, UpdatePizzaOp):
            _validate_index(op.index, len(pizzas))
            current = pizzas[op.index]
            updated = current.model_copy(update=op.value.model_dump(exclude_none=True))
            pizzas[op.index] = updated
        elif isinstance(op, RemovePizzaOp):
            _validate_index(op.index, len(pizzas))
            del pizzas[op.index]
        elif isinstance(op, MovePizzaOp):
            _validate_index(op.from_index, len(pizzas))
            _validate_index(op.to_index, len(pizzas), allow_end=True)
            item = pizzas.pop(op.from_index)
            pizzas.insert(op.to_index, item)
        else:
            raise ValueError(f"Unsupported op: {op}")

    return doc.model_copy(update={"pizzas": pizzas})


In [41]:
def _build_patch_prompt(*, doc: PizzaOrderDoc, instruction: str, error: str | None = None) -> str:
    base = (
        "You are editing a list of pizzas. Generate a PizzaEditPlan JSON that applies only the necessary edits.\n"
        "Rules:\n"
        "- Use move when reordering (do not remove + add for moves).\n"
        "- Use update for field changes; only include fields that changed.\n"
        "- Use add with a full Pizza object.\n"
        "- Indices refer to the CURRENT list state and operations are applied sequentially.\n"
        "- Keep the plan minimal.\n"
    )
    if error:
        base += f"Previous attempt failed with error: {error}\n"
    base += f"Current pizzas (index -> pizza):\n{doc.model_dump()}\n\nInstruction:\n{instruction}"
    return base


def request_pizza_plan(*, doc: PizzaOrderDoc, instruction: str, max_attempts: int = 2) -> PizzaOrderDoc:
    schema = PizzaEditPlan.model_json_schema()
    last_error: str | None = None
    for _ in range(max_attempts):
        prompt = _build_patch_prompt(doc=doc, instruction=instruction, error=last_error)
        response = client.responses.create(
            model=model,
            input=[{"role": "user", "content": prompt}],
            text={
                "format": {
                    "type": "json_schema",
                    "name": "pizza_edit_plan",
                    "strict": True,
                    "schema": schema,
                }
            },
        )
        plan = PizzaEditPlan.model_validate_json(response.output_text)
        try:
            return apply_pizza_plan(doc, plan)
        except Exception as exc:
            last_error = str(exc)
            continue
    raise RuntimeError(f"Failed to apply patch plan after {max_attempts} attempts: {last_error}")


In [42]:
current = PizzaOrderDoc(
    pizzas=[
        Pizza(name="pepperoni", add=["olives"]),
        Pizza(name="margherita"),
    ]
)

instruction = "Move margherita to index 0 and add a veggie pizza with mushrooms."
updated = request_pizza_plan(doc=current, instruction=instruction, max_attempts=2)
updated


PizzaOrderDoc(pizzas=[Pizza(name='margherita', remove=None, add=None), Pizza(name='pepperoni', remove=None, add=['olives']), Pizza(name='veggie', remove=None, add=['mushrooms'])])

PizzaOrder(size='large', toppings=['pepperoni', 'mushrooms'], address='123 Main St.')